In [1]:
import pickle
import numpy as np

test_img = pickle.load(open('pickle_test_img.p', 'rb'))
test_img = np.squeeze(test_img, axis = -1)
test_img.shape

#training_img_1 = pickle.load(open('processed_train_img_1.p', 'rb'))
#training_img_2 = pickle.load(open('processed_train_img_2.p', 'rb'))

(11212, 256, 256)

In [2]:
#np.set_printoptions(threshold=np.nan, linewidth=115)

#training_img = np.append(training_img_1, training_img_2, axis = 0)

reshape_array = np.reshape(test_img, (11212, 256 * 256))
reshape_array.shape

(11212, 65536)

In [3]:
test_labels = pickle.load(open('test_labels.p', 'rb'))
test_labels = test_labels.astype(dtype='uint8')
test_labels.shape

#training_labels = pickle.load(open('training_labels.p', 'rb'))
#training_labels = training_labels.astype(dtype='uint8')

#print(training_labels[:10])
#training_labels.shape

(11212, 15)

In [4]:
Infiltration_test_labels = test_labels[:, 8]
Infiltration_test_labels.shape

(11212,)

In [6]:
Infiltration_test_dict = {0: 1745, 1: 1745}

from imblearn.under_sampling import RandomUnderSampler

RUS = RandomUnderSampler(ratio = Infiltration_test_dict)

test_img_res, test_label_res = RUS.fit_sample(reshape_array, Infiltration_test_labels)

In [7]:
vec_int_training_labels = np.array(test_label_res, dtype='uint8')

vec_int_training_labels.shape

(3490,)

In [9]:
test_img_new = test_img_res.reshape((len(vec_int_training_labels), 256, 256))

In [10]:
test_img = np.expand_dims(test_img_new, axis = 3)
test_img.shape

(3490, 256, 256, 1)

In [11]:
# Save Processed Images
pickle.dump(test_img, open('1vAll_test_img.p', 'wb'))

In [12]:
pickle.dump(vec_int_training_labels, open('1vAll_test_labels.p', 'wb'))

### Load Processed Images

In [ ]:
# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('training_img_1.p', 'rb'))
training_img_2 = pickle.load(open('training_img_2.p', 'rb'))
training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('pickle_test_img.p', 'rb'))

#training_labels = pickle.load(open('training_labels.p', 'rb'))
#test_labels = pickle.load(open('test_labels.p', 'rb'))

# Neural Network

In [ ]:
import keras
from keras import models, optimizers, layers
from keras.models import model_from_json
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

In [ ]:
baseline = models.Sequential()
baseline.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
baseline.add(layers.MaxPooling2D((2, 2)))
baseline.add(layers.Conv2D(64, (3, 3), activation='relu'))
baseline.add(layers.MaxPooling2D((2, 2)))
baseline.add(layers.Conv2D(64, (3, 3), activation='relu'))
baseline.add(layers.MaxPooling2D((2, 2)))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(64, activation='relu'))
baseline.add(layers.Dense(15, activation='sigmoid'))

#baseline.summary()

In [ ]:
baseline.compile(optimizer = optimizers.RMSprop(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
base_history = baseline.fit(training_img, training_labels, epochs = 40, validation_split = (1 / 9), batch_size = 128, verbose = 1)

In [ ]:
save_history(base_history)

#### Save Base Model

In [ ]:
save_base(base_history)

In [ ]:
baseline_test = baseline.evaluate(test_img, test_labels)
print(baseline_test)

In [ ]:
import matplotlib.pyplot as plt

acc = base_history.history['acc']
val_acc = base_history.history['val_acc']
loss = base_history.history['loss']
val_loss = base_history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Comparison Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model_4C_4MP_1D = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1)

In [ ]:
model_test = model.evaluate(test_img, test_labels)

In [ ]:
print(model_test)

In [ ]:
save_history(model_4C_4MP_1D)

### Plot Results Compared To Baseline

In [ ]:
base_acc = bases_loaded.history['val_acc']
model_acc = model_4C_4MP_1D.history['val_acc']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, base_acc, 'b', label='Base Acc')
plt.plot(epochs, model_acc, 'r', label='Model Acc')
plt.title('Comparison of Validation Accuracies')
plt.legend()
plt.show()

In [ ]:
base_loss = bases_loaded.history['val_loss']
model_loss = model_4C_4MP_1D.history['val_loss']
plt.plot(epochs, base_loss, 'b', label='Base loss')
plt.plot(epochs, model_loss, 'r', label='Model loss')
plt.title('Comparison of Validation Losses')
plt.legend()
plt.show()

### Split Labels

In [ ]:
read_data = pd.read_csv(TRAIN_LABEL_DIR)
split_labels = [items.split('|') for items in read_data.Finding_Labels]

all_img = np.append(test_img, training_img, axis = 0)
print(all_img.shape)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#numpy.set_printoptions(threshold=numpy.nan)
str_array = np.hstack(split_labels)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(str_array)
#print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse = False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded[:10])

onehot_encoded = onehot_encoded.astype('int32')

In [ ]:
print(onehot_encoded.shape)
print(onehot_encoded.ndim)

In [ ]:
np.expand_dims(onehot_encoded, 0)
print(onehot_encoded.ndim)
print(onehot_encoded.shape)

In [ ]:
print(len(str_array))
print(len(onehot_encoded))

In [ ]:
flattened_data = []
index = 0
for i in range(0, len(all_img)):
    for label in split_labels[i]:
        flattened_data.append((onehot_encoded[index], all_img[i]))
        index += 1

In [ ]:
np.set_printoptions(threshold = np.nan, linewidth = 115)

In [ ]:
#print(flattened_data[1])
final_img = []

i = 0
while i < len(flattened_data):
    final_img.append(flattened_data[i][1])
    i += 1

In [ ]:
len(final_img)

In [ ]:
final_img = np.array(final_img[:(len(final_img)/3)])

In [ ]:
print(flattened_data[3][1])

In [ ]:
print(len(flattened_data))